In [ ]:
import pandas as pd

In [11]:
# Load the new Excel file
df_wf = pd.read_excel('employee_org_structure.xlsx', engine='openpyxl')
df_wf.head()

,Emp ID,Name,Position,Level 1 Manager ID,Level 1 Manager Name,Level 2 Manager ID,Level 2 Manager Name,Level 3 Manager ID,Level 3 Manager Name
0,1,Alice,CEO,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Bob,CTO,1.0,Alice,NaN,NaN,NaN,NaN
2,3,Charlie,CFO,1.0,Alice,NaN,NaN,NaN,NaN
3,4,David,Manager,3.0,Charlie,1.0,Alice,NaN,NaN
4,5,Eve,Analyst,3.0,Charlie,1.0,Alice,NaN,NaN


# Method 1 - Iterating through columns

In [ ]:
# Create a new list to store the transformed data
long_format = []

# number of manager levels
num_levels = 3

# Iterate through each row in the original DataFrame
for index, row in df_wf.iterrows():
    # check each manager level
    for i in range(1, num_levels + 1):

        # Get the manager details at each level
        manager_id = f'Level {i} Manager ID'
        manager_name = f'Level {i} Manager Name'

        # skip if the manager is not available at this level, except if level 1
        if pd.isna(row[manager_id]) and i > 1:
            break
        
        #append each manager level to the list
        long_format.append({
        'Emp ID': row['Emp ID'],
        'Name': row['Name'],
        'Position': row['Position'],
        'Manager Level': i,
        'Manager ID': row[manager_id],
        'Manager Name': row[manager_name]
        })
    
# Convert the list into a DataFrame
df_lf = pd.DataFrame(long_format)

# write the data frame to an excel sheet
df_lf.to_excel("org_structure_long_format_method_1.xlsx", index=False)

df_lf.head()

# Method 2 - Using melt

In [ ]:
# the columns that do not need to be unpivoted
id_vars = ['Emp ID', 'Name', 'Position']

#rest of columns that need to be unpivoted
value_vars = df_wf.columns[3:]  

# Unpivot the DataFrame
df_lf2 = df_wf.melt(id_vars=id_vars, 
                       value_vars=value_vars, 
                       var_name='manager_level_and_type', 
                       value_name='manager_info')

# new column converting 'Level X Manager ID' or 'Level X Manager Name' to just the number
df_lf2['Level'] = df_lf2['manager_level_and_type'].str.extract(r'Level (\d+)')

#column indicating if the row contains the manger name or the ID
df_lf2['Type'] = df_lf2['manager_level_and_type'].str.extract(r'(ID|Name)')

#new columns to store manager id and manager name seperately.
df_lf2['manager_id'] = df_lf2.apply(lambda x: x['manager_info'] if x['Type'] == 'ID' else None, axis=1)
df_lf2['manager_name'] = df_lf2.apply(lambda x: x['manager_info'] if x['Type'] == 'Name' else None, axis=1)

# removing unwanted columns
df_lf2.drop(columns=['manager_level_and_type','manager_info'], inplace=True)

#creating two new dataframes - one with only manager id and one with manager name, then merging them
df_lf2['lookup'] = df_lf2['Emp ID'].astype('str') + "-" + df_lf2['Level'].astype('str') #column that we will use for merging two columns
df_name = df_lf2[df_lf2['Type'] == 'Name'].drop(columns=['manager_id','Type'])
df_id = df_lf2[df_lf2['Type'] == 'ID'].drop(columns=['manager_name','Type'])
df_lf2 = df_id.merge(df_name[['lookup','manager_name']],on='lookup').drop(columns='lookup').sort_values(['Emp ID','Level'])

#logic to drop rows with no manager id and name except when level is 1
df = df_lf2[df_lf2['Level'] != '1'].dropna()
df2 = df_lf2[df_lf2['Level'] == '1']
df_lf2 = pd.concat([df,df2]).sort_values(['Emp ID','Level'])

# write the data frame to an excel sheet
df_lf2.to_excel("org_structure_long_format_method_2.xlsx", index=False)

df_lf2.head()

,Emp ID,Name,Position,Level,manager_id,manager_name
0,1,Alice,CEO,1,NaN,NaN
1,2,Bob,CTO,1,1.0,Alice
2,3,Charlie,CFO,1,1.0,Alice
3,4,David,Manager,1,3.0,Charlie
109,4,David,Manager,2,1.0,Alice
